## Problem:

- In out problem, need to detect the nearest pedestrian road to see if the car is violating or not 
- need to know which part is the pedestrian road and which part is the car road  

In [1]:
import sys
import cv2
import time
import numpy as np
import ffmpeg
import matplotlib.pyplot as plt
from pathlib import Path

repo_path = Path(".").absolute().parent
# data_path = Path(".") / "videos"
data_path = Path("D:\Datas\parking_violation")
pkg_path = repo_path.parent / "LaneATT"
cfg_path = pkg_path / "cfgs" / "test.yml"

sys.path.append(str(repo_path))
sys.path.append(str(pkg_path))

In [2]:
import torch
import torch.nn as nn
from lanedetection import imreader, imsaver
from lib.config import Config
from lib.models import LaneATT
from lib.runner import Runner
from lib.datasets.lane_dataset import LaneDataset
from lanedetection import check_rotation, correct_rotation, correct_rgb, resize_image

height, width = 360, 640

In [3]:
cfg = Config(cfg_path)
device = torch.device('cpu') # if not torch.cuda.is_available() or args.cpu else torch.device('cuda')
runner = Runner(cfg, "test", device, view="all", resume=False, deterministic=False)

In [48]:
video_path = str(data_path / "sample1.mp4")
sv_path = repo_path / "notebooks" / "imgs" / "sample1_fps30"

cap = cv2.VideoCapture(video_path)
rotate_code = check_rotation(video_path)

frame_rate = 30
prev = 0
count = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    time_elapsed = time.time() - prev
    if ret & (time_elapsed > 1./frame_rate):
        prev = time.time()
        # process image
        if rotate_code:
            frame = correct_rotation(frame, rotate_code)
        frame = resize_image(frame, width, height)
        # frame = correct_rgb(frame)
        # pipeline
        images = frame
        
        cv2.imwrite( str(sv_path / f"{count:05d}.jpg"), images)
        count += 1
        
        cv2.imshow("video", images)
        key = cv2.waitKey(5)
        if key == ord("q"):
            cv2.destroyAllWindows()
            break
    if not ret:
        break
            
if not ret:
    cv2.destroyAllWindows()

In [4]:
img_path = "./imgs/test2.jpg"
img = imreader(img_path)

In [8]:
# Dataset
S = cfg.config["datasets"]["test"]["parameters"]["S"]
normalize = False
img_size = cfg.config["datasets"]["test"]["parameters"]["img_size"]
n_strips = S - 1
n_offsets = S
normalize = normalize
img_h, img_w = img_size
strip_size = img_h / n_strips

In [23]:
# backbone="resnet34"
# pretrained_backbone=True
# S=72
# img_w=width
# img_h=height
# anchors_freq_path= str(pkg_path / "data" / "culane_anchors_freq.pt")
# topk_anchors=1000
# anchor_feat_channels=64
model_path = pkg_path / "experiments" / "laneatt_r34_culane" / "models" / "model_0015.pt"
model = cfg.get_model()
model.load_state_dict(torch.load(model_path)["model"])

<All keys matched successfully>

In [32]:
inputs = torch.FloatTensor(img).permute(2, 0, 1).unsqueeze(0)
test_parameters = cfg.get_test_parameters()
conf_threshold, nms_thres, nms_topk = test_parameters.values()

In [25]:
outputs = model(inputs, **test_parameters)
prediction = model.decode(outputs, as_lanes=True)

C:\Users\simon\miniconda3\envs\venv\lib\site-packages\nms\malisiewicz.py:66: RuntimeWarning: divide by zero encountered in true_divide
  return (w * h)/area


ValueError: not enough values to unpack (expected 3, got 1)

In [40]:
from nms import nms

In [42]:
from nms.nms import boxes, rboxes, polygons

In [43]:
boxes

<function nms.nms.boxes(rects, scores, nms_algorithm=<function nms at 0x0000022B1FE8E158>, **kwargs)>

In [32]:
proposals_list = model.nms(reg_proposals, attention_matrix, nms_thres, nms_topk, conf_threshold)

AssertionError: 

In [48]:
x = torch.FloatTensor(img).permute(2, 0, 1).unsqueeze(0)

batch_features = model.feature_extractor(x)
batch_features = model.conv1(batch_features)
batch_anchor_features = model.cut_anchor_features(batch_features)

# Join proposals from all images into a single proposals features batch
batch_anchor_features = batch_anchor_features.view(-1, model.anchor_feat_channels * model.fmap_h)

# Add attention features
softmax = nn.Softmax(dim=1)
scores = model.attention_layer(batch_anchor_features)
attention = softmax(scores).reshape(x.shape[0], len(model.anchors), -1)
attention_matrix = torch.eye(attention.shape[1], device=x.device).repeat(x.shape[0], 1, 1)
non_diag_inds = torch.nonzero(attention_matrix == 0., as_tuple=False)
attention_matrix[:] = 0
attention_matrix[non_diag_inds[:, 0], non_diag_inds[:, 1], non_diag_inds[:, 2]] = attention.flatten()
batch_anchor_features = batch_anchor_features.reshape(x.shape[0], len(model.anchors), -1)
attention_features = torch.bmm(torch.transpose(batch_anchor_features, 1, 2),
                               torch.transpose(attention_matrix, 1, 2)).transpose(1, 2)
attention_features = attention_features.reshape(-1, model.anchor_feat_channels * model.fmap_h)
batch_anchor_features = batch_anchor_features.reshape(-1, model.anchor_feat_channels * model.fmap_h)
batch_anchor_features = torch.cat((attention_features, batch_anchor_features), dim=1)

# Predict
cls_logits = model.cls_layer(batch_anchor_features)
reg = model.reg_layer(batch_anchor_features)

# Undo joining
cls_logits = cls_logits.reshape(x.shape[0], -1, cls_logits.shape[1])
reg = reg.reshape(x.shape[0], -1, reg.shape[1])

# Add offsets to anchors
reg_proposals = torch.zeros((*cls_logits.shape[:2], 5 + model.n_offsets), device=x.device)
reg_proposals += model.anchors
reg_proposals[:, :, :2] = cls_logits
reg_proposals[:, :, 4:] += reg

In [50]:
reg_proposals.shape, attention_matrix.shape

(torch.Size([1, 1000, 77]), torch.Size([1, 1000, 1000]))

In [51]:
batch_proposals = reg_proposals
batch_attention_matrix = attention_matrix

softmax = nn.Softmax(dim=1)
proposals_list = []
for proposals, attention_matrix in zip(batch_proposals, batch_attention_matrix):
    break

In [52]:
proposals.shape, attention_matrix.shape

(torch.Size([1000, 77]), torch.Size([1000, 1000]))

In [59]:
anchor_inds = torch.arange(batch_proposals.shape[1], device=proposals.device)
scores = softmax(proposals[:, :2])[:, 1]

In [66]:
above_threshold = scores > conf_threshold
proposals = proposals[above_threshold]

In [68]:
scores = scores[above_threshold]
anchor_inds = anchor_inds[above_threshold]

In [73]:
boxes(proposals.detach(), scores, overlap=nms_thres, top_k=nms_topk)

[7]

In [78]:
proposals.shape

torch.Size([9, 77])

In [74]:
scores

tensor([0.5288, 0.6722, 0.6807, 0.6726, 0.6786, 0.6786, 0.5061, 0.5362, 0.5180],
       grad_fn=<IndexBackward>)

In [ ]:
nms.()

In [44]:
batch_proposals = reg_proposals
batch_attention_matrix = attention_matrix

softmax = nn.Softmax(dim=1)
proposals_list = []
for proposals, attention_matrix in zip(batch_proposals, batch_attention_matrix):
    anchor_inds = torch.arange(batch_proposals.shape[1], device=proposals.device)
    # The gradients do not have to (and can't) be calculated for the NMS procedure
    with torch.no_grad():
        scores = softmax(proposals[:, :2])[:, 1]
        if conf_threshold is not None:
            # apply confidence threshold
            above_threshold = scores > conf_threshold
            proposals = proposals[above_threshold]
            scores = scores[above_threshold]
            anchor_inds = anchor_inds[above_threshold]
        if proposals.shape[0] == 0:
            proposals_list.append((proposals[[]], model.anchors[[]], attention_matrix[[]], None))
            continue
        keep, num_to_keep, _ = boxes(proposals, scores, overlap=nms_thres, top_k=nms_topk)
        keep = keep[:num_to_keep]
    proposals = proposals[keep]
    anchor_inds = anchor_inds[keep]
    attention_matrix = attention_matrix[anchor_inds]
    proposals_list.append((proposals, model.anchors[keep], attention_matrix, anchor_inds))


TypeError: zip argument #2 must support iteration

In [37]:
proposals.shape

torch.Size([9, 77])

In [38]:
scores.shape

torch.Size([9])

In [ ]:
img = (images[0].cpu().permute(1, 2, 0).numpy() * 255).astype(np.uint8)
img, fp, fn = dataloader.dataset.draw_annotation(idx, img=img, pred=prediction[0])
if self.view == 'mistakes' and fp == 0 and fn == 0:
    continue
cv2.imshow('pred', img)
cv2.waitKey(0)